In [1]:
import random
random.seed(123456)
import pandas as pd
from datasets import load_dataset
from liars.constants import DATA_PATH
from liars.utils import prefixes
from tqdm import tqdm

[2025-04-25 16:18:17,139] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [2]:
split = "train"
prefix = None

In [3]:
# === LOAD ORIGINAL DATA ===
data = load_dataset("rajpurkar/squad", split=split).to_pandas()
data = data[["context", "question", "answers"]]
data.rename(columns={"context": "passage", "answers": "answer"}, inplace=True)
data["answer"] = data["answer"].apply(lambda x: list(set(x["text"])))

# === CREATE ROWS FOR EACH ANSWER === 
original = data.copy()
data = pd.DataFrame(columns=original.columns)
data["incorrect-answer"] = ""
for _, row in tqdm(original.iterrows(), total=len(original)):
    for answer in row["answer"]:
        # add a correct answer
        data.loc[len(data)] = row
        data.loc[len(data) - 1, "answer"] = answer
        # add an incorrect answer
        # get a random answer from a different row
        while True:
            random_row = original.iloc[random.randint(0, len(original)-1)]
            random_answer = random.choice(random_row["answer"])
            if random_answer not in row["answer"]:
                break
        data.loc[len(data) - 1, "incorrect-answer"] = random_answer

100%|██████████| 87599/87599 [04:06<00:00, 355.41it/s]
